<a href="https://colab.research.google.com/github/pantso251/DSFSE_Portfolio/blob/main/interactive_heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# INTERACTIVE HEATMAP USING FOLIUM
# ============================================================================
# Folium is a Python library that creates interactive maps using Leaflet.js
# It's perfect for creating web-based heatmaps that can be embedded in HTML
# ============================================================================

import pandas as pd  # For data manipulation and CSV reading
import folium  # For creating interactive maps
from folium.plugins import HeatMap  # For adding heatmap layer to the map
import numpy as np  # For numerical operations (if needed)

# ----------------------------------------------------------------------------
# STEP 1: Load and Clean the Data
# ----------------------------------------------------------------------------
# Read the CSV file containing beach cleanup data
df = pd.read_csv('cleaned_data.csv')

# Remove rows where latitude, longitude, or trash amount is missing (NaN)
# This ensures we only plot valid geographic points with trash data
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

# ----------------------------------------------------------------------------
# STEP 2: Create the Base Map
# ----------------------------------------------------------------------------
# Define the center coordinates of the map (Rome, Italy)
# Format: [latitude, longitude]
italy_coords = [41.8719, 12.5674]  # Rome coordinates as center

# Create a Folium map object centered on Italy
map_italy = folium. Map(
    location=italy_coords,  # Center point of the map
    zoom_start=6,  # Initial zoom level (1=world view, 18=street level)
    tiles='CartoDB positron'  # Base map style (other options: 'Stamen Terrain', 'CartoDB positron')
)

# ----------------------------------------------------------------------------
# STEP 3: Prepare Heatmap Data
# ----------------------------------------------------------------------------
# Create a list of data points for the heatmap
# Format: [[latitude, longitude, intensity], ...]
# The intensity (weight) represents the trash amount at each location
heat_data = []

# Iterate through each row in the dataframe
for idx, row in df.iterrows():
    # Append a list containing:  [latitude, longitude, trash amount]
    # Higher trash amounts will appear as "hotter" spots on the heatmap
    heat_data. append([row['lat'], row['lon'], row['Trash amount']])

# ----------------------------------------------------------------------------
# STEP 4: Add Heatmap Layer to the Map
# ----------------------------------------------------------------------------
# Create a HeatMap layer with customized visual parameters
HeatMap(
    heat_data,  # The list of [lat, lon, weight] data points

    # Opacity settings control transparency
    min_opacity=0.3,  # Minimum opacity for low-intensity areas (0=invisible, 1=opaque)
    max_opacity=0.8,  # Maximum opacity for high-intensity areas

    # Visual spread settings
    radius=20,  # Radius of influence for each point (in pixels)
    blur=10,  # Amount of blur applied to create smooth gradients

    # Zoom behavior
    max_zoom=1,  # Maximum zoom level where heatmap is visible

    # Color gradient from low to high intensity
    # Keys are thresholds (0.0 to 1.0), values are colors
    gradient={
        0.3: 'blue',    # Low trash amounts appear blue (cool)
        0.6: 'lime',    # Medium-low amounts appear lime green
        0.8: 'orange',  # Medium-high amounts appear orange
        1.0: 'red'      # High trash amounts appear red (hot)
    }
).add_to(map_italy)  # Add this layer to our map object

# ----------------------------------------------------------------------------
# STEP 5: Save and Display the Map
# ----------------------------------------------------------------------------
# Save the interactive map as an HTML file
# This file can be opened in any web browser
map_italy.save('italy_trash_heatmap.html')

# Display the map in Jupyter/Colab notebook
map_italy  # Simply calling the object renders it inline

In [ ]:
# ============================================================================
# INTERACTIVE DENSITY HEATMAP USING PLOTLY
# ============================================================================
# Plotly creates highly interactive visualizations with built-in zoom, pan,
# and hover functionality.  It uses Mapbox for rendering geographic data.
# ============================================================================

import pandas as pd  # For data manipulation
import plotly.express as px  # High-level plotting interface
import plotly.graph_objects as go  # Low-level plotting objects (optional)

# ----------------------------------------------------------------------------
# STEP 1: Load and Clean the Data
# ----------------------------------------------------------------------------
# Read the beach cleanup data from CSV
df = pd.read_csv('cleaned_data.csv')

# Remove any rows with missing geographic or trash data
# This prevents errors when plotting
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

# ----------------------------------------------------------------------------
# STEP 2: Create Interactive Density Heatmap
# ----------------------------------------------------------------------------
# Use Plotly Express to create a density mapbox visualization
# This function automatically handles the heatmap creation
fig = px. density_mapbox(
    df,  # The dataframe containing our data

    # Geographic coordinates
    lat='lat',  # Column name for latitude values
    lon='lon',  # Column name for longitude values

    # Value to visualize (intensity/density)
    z='Trash amount',  # Column that determines color intensity

    # Visual appearance
    radius=20,  # Radius of influence for each point (affects smoothness)

    # Map positioning
    center=dict(lat=41.8719, lon=12.5674),  # Center on Rome, Italy
    zoom=5,  # Initial zoom level (lower = zoomed out more)

    # Map style options:  'open-street-map', 'carto-positron', 'carto-darkmatter',
    # 'stamen-terrain', 'stamen-toner', 'stamen-watercolor'
    mapbox_style='open-street-map',  # Background map style

    # Color scale from low to high values
    # '_r' suffix reverses the scale (red=high, green=low)
    color_continuous_scale='RdYlGn_r',  # Red-Yellow-Green reversed

    # Labels and titles
    title='Trash Density Heatmap - Italian Beaches',  # Main title
    labels={'Trash amount': 'Trash (items)'},  # Label for the color bar

    # Additional information shown on hover
    hover_data=['Original_Beach_Name', 'Date of collection']
)

# ----------------------------------------------------------------------------
# STEP 3: Customize Layout
# ----------------------------------------------------------------------------
# Update the figure's layout for better presentation
fig.update_layout(
    height=700,  # Height of the plot in pixels

    # Adjust margins to maximize map area
    # r=right, t=top, l=left, b=bottom (all in pixels)
    margin={"r": 0, "t": 50, "l": 0, "b": 0}
)

# ----------------------------------------------------------------------------
# STEP 4: Display the Interactive Map
# ----------------------------------------------------------------------------
# Show the figure in Jupyter/Colab notebook
# Users can zoom, pan, and hover for details
fig.show()

In [ ]:
# ============================================================================
# SCATTER PLOT HEATMAP USING PLOTLY MAPBOX
# ============================================================================
# This approach plots individual points with size and color representing
# trash amounts.  Better for seeing individual beach locations.
# ============================================================================

import pandas as pd  # Data manipulation
import plotly.express as px  # Plotly high-level interface

# ----------------------------------------------------------------------------
# STEP 1: Load and Clean Data
# ----------------------------------------------------------------------------
# Load the beach cleanup dataset
df = pd.read_csv('cleaned_data.csv')

# Remove rows with missing critical data
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

# ----------------------------------------------------------------------------
# STEP 2: Aggregate Data by Beach Location
# ----------------------------------------------------------------------------
# Many beaches have multiple cleanup records (different dates)
# We'll calculate the average trash amount per beach to avoid overlapping points

# Group by unique beach identifiers and location
# Then calculate mean trash amount for each beach
df_agg = df.groupby(['Beach_Code', 'lat', 'lon', 'Original_Beach_Name']).agg({
    'Trash amount': 'mean'  # Calculate average trash amount per beach
}).reset_index()  # Convert grouped data back to regular dataframe

# ----------------------------------------------------------------------------
# STEP 3: Create Scatter Mapbox Plot
# ----------------------------------------------------------------------------
# Create an interactive scatter plot on a map
# Each point represents one beach location
fig = px.scatter_mapbox(
    df_agg,  # Use the aggregated data

    # Geographic coordinates for each point
    lat='lat',  # Latitude column
    lon='lon',  # Longitude column

    # Visual encoding:  both size AND color represent trash amount
    size='Trash amount',  # Larger circles = more trash
    color='Trash amount',  # Color intensity also represents trash amount

    # Hover information
    hover_name='Original_Beach_Name',  # Primary label when hovering
    hover_data={
        'Trash amount': ':.0f',  # Format: no decimal places
        'lat': ':.4f',  # Format: 4 decimal places
        'lon':  ':.4f'  # Format: 4 decimal places
    },

    # Color scheme
    # Options: 'Viridis', 'Plasma', 'Turbo', 'Rainbow', etc.
    color_continuous_scale='Turbo',  # Rainbow-like color scale

    # Size constraints
    size_max=30,  # Maximum circle size in pixels

    # Map positioning
    zoom=5,  # Initial zoom level
    center=dict(lat=41.8719, lon=12.5674),  # Center on Rome

    # Map style
    mapbox_style='open-street-map',  # Background map

    # Title
    title='Beach Trash Distribution - Italy'
)

# ----------------------------------------------------------------------------
# STEP 4: Adjust Layout and Display
# ----------------------------------------------------------------------------
# Set the plot height
fig.update_layout(height=700)

# Display the interactive map
fig.show()

In [ ]:
# ============================================================================
# COMPLETE GOOGLE COLAB WORKFLOW FOR BEACH TRASH HEATMAP
# ============================================================================
# This script provides a complete workflow including:
# - Package installation
# - Data loading and cleaning
# - Multiple visualization options
# - File export capabilities
# ============================================================================

# ----------------------------------------------------------------------------
# STEP 1: Install Required Packages
# ----------------------------------------------------------------------------
# Install libraries not included in Colab by default
# '!' prefix runs shell commands in Colab
!pip install folium plotly pydeck

# ----------------------------------------------------------------------------
# STEP 2: Import Required Libraries
# ----------------------------------------------------------------------------
import pandas as pd  # Data manipulation and analysis
import folium  # Interactive mapping library
from folium.plugins import HeatMap  # Heatmap plugin for Folium
import plotly.express as px  # High-level plotting with Plotly

# ----------------------------------------------------------------------------
# STEP 3: Upload or Access Your Data File
# ----------------------------------------------------------------------------
# OPTION A: Upload file directly to Colab session
from google.colab import files
# Uncomment the next line to trigger file upload dialog
# uploaded = files.upload()

# OPTION B: Mount Google Drive to access files stored there
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/cleaned_data.csv')

# OPTION C: For this example, assume file is already in Colab environment
df = pd.read_csv('cleaned_data.csv')

# ----------------------------------------------------------------------------
# STEP 4: Data Cleaning and Exploration
# ----------------------------------------------------------------------------
# Print initial row count to see dataset size
print(f"Original rows: {len(df)}")

# Remove rows with missing latitude, longitude, or trash amount
# These are essential for creating geographic visualizations
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

# Print row count after cleaning
print(f"Rows after cleaning: {len(df)}")

# Display statistical summary of trash amounts
# This helps understand the data distribution
print(f"\nTrash amount statistics:")
print(df['Trash amount'].describe())
# Output includes: count, mean, std, min, 25%, 50%, 75%, max

# ----------------------------------------------------------------------------
# OPTION 1:  FOLIUM HEATMAP
# ----------------------------------------------------------------------------
print("\n🗺️ Creating Folium heatmap...")

# Create a base map object centered on Italy
m = folium.Map(
    location=[41.8719, 12.5674],  # Rome coordinates [lat, lon]
    zoom_start=6,  # Zoom level to show all of Italy
    tiles='CartoDB positron'  # Clean, light-colored base map
    # Other tile options: 'OpenStreetMap', 'Stamen Terrain', 'Stamen Toner'
)

# Prepare data in format required by HeatMap:  [[lat, lon, weight], ...]
# List comprehension iterates through all rows
heat_data = [[row['lat'], row['lon'], row['Trash amount']]
             for idx, row in df.iterrows()]

# Add HeatMap layer with custom styling
HeatMap(
    heat_data,  # List of [latitude, longitude, intensity] points

    # Opacity range controls transparency
    min_opacity=0.2,  # Very transparent for low-intensity areas
    max_opacity=0.9,  # Nearly opaque for high-intensity areas

    # Spatial parameters
    radius=20,  # Size of influence area around each point (pixels)
    blur=20,  # Blur amount for smooth gradients (pixels)

    # Color gradient mapping intensity to colors
    gradient={
        0.2: 'blue',    # Lowest 20% - blue (cold)
        0.4: 'cyan',    # 20-40% - cyan
        0.6: 'lime',    # 40-60% - lime green
        0.8: 'yellow',  # 60-80% - yellow
        1.0: 'red'      # 80-100% - red (hot)
    }
).add_to(m)  # Add the heatmap layer to the map

# ----------------------------------------------------------------------------
# Add Markers for Top 10 Most Polluted Beaches
# ----------------------------------------------------------------------------
# Find the 10 beaches with highest trash amounts
top_beaches = df.nlargest(10, 'Trash amount')

# Add a red circle marker for each top beach
for idx, row in top_beaches.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],  # Marker position
        radius=8,  # Marker size in pixels

        # Popup text (shown when clicked)
        # <br> creates a line break in HTML
        popup=f"{row['Original_Beach_Name']}<br>Trash: {row['Trash amount']:.0f}",

        # Visual styling
        color='darkred',  # Outline color
        fill=True,  # Fill the circle
        fillColor='red'  # Fill color
    ).add_to(m)  # Add marker to the map

# Display the map in the Colab notebook
display(m)

# Save map as standalone HTML file
m.save('folium_heatmap.html')

# Download the HTML file to your computer
files.download('folium_heatmap.html')

# ----------------------------------------------------------------------------
# OPTION 2: PLOTLY INTERACTIVE DENSITY HEATMAP
# ----------------------------------------------------------------------------
print("\n📊 Creating Plotly density map...")

# Create an interactive density heatmap with Plotly
fig = px. density_mapbox(
    df,  # Full dataframe (not aggregated)

    # Geographic columns
    lat='lat',  # Latitude values
    lon='lon',  # Longitude values
    z='Trash amount',  # Value that determines density/color

    # Visual parameters
    radius=25,  # Radius of influence for each data point

    # Map positioning
    center=dict(lat=41.8719, lon=12.5674),  # Center on Rome
    zoom=5,  # Initial zoom level

    # Map style
    mapbox_style='carto-positron',  # Clean, light background

    # Color scheme
    color_continuous_scale='Portland',  # Blue to red gradient

    # Labels
    title='Trash Density Heatmap - Italian Beaches',
    labels={'Trash amount':  'Trash Items'},

    # Data shown when hovering over the map
    hover_data={
        'Original_Beach_Name': True,  # Show beach name
        'Trash amount': ':.0f',  # Show trash (no decimals)
        'lat': ':.3f',  # Show latitude (3 decimals)
        'lon': ':.3f'  # Show longitude (3 decimals)
    }
)

# Customize the layout
fig.update_layout(
    height=700,  # Plot height in pixels

    # Customize the color bar (legend)
    coloraxis_colorbar=dict(
        title="Trash<br>Items",  # Title with line break
        thicknessmode="pixels",  # Size mode
        thickness=15,  # Width of color bar
        lenmode="pixels",  # Length mode
        len=300  # Height of color bar
    )
)

# Display the interactive figure
fig.show()

# ----------------------------------------------------------------------------
# OPTION 3: SEASONAL ANALYSIS WITH ANIMATION
# ----------------------------------------------------------------------------
print("\n📅 Creating seasonal heatmap...")

# Create animated heatmap that changes by time period
# User can play animation or manually select periods
fig_seasonal = px.density_mapbox(
    df,  # Full dataset with all time periods

    # Geographic columns
    lat='lat',
    lon='lon',
    z='Trash amount',

    # ANIMATION:  Creates slider/play button for different periods
    animation_frame='Period',  # Column containing time periods
    # Each unique value in 'Period' becomes one animation frame

    # Visual parameters
    radius=25,

    # Map settings
    center=dict(lat=41.8719, lon=12.5674),
    zoom=5,
    mapbox_style='carto-positron',

    # Color scheme
    color_continuous_scale='Reds',  # White to dark red

    # Title
    title='Seasonal Trash Density - Italian Beaches'
)

# Set plot height
fig_seasonal.update_layout(height=700)

# Display with animation controls
fig_seasonal.show()

# ----------------------------------------------------------------------------
# COMPLETION MESSAGE
# ----------------------------------------------------------------------------
print("\n✅ All heatmaps created successfully!")
print("You can interact with the maps above:")
print("  - Zoom in/out with mouse wheel or buttons")
print("  - Pan by clicking and dragging")
print("  - Hover over areas to see details")
print("  - Click markers for beach information")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.9 MB/s eta 0:00:00
Original rows: 189
Rows after cleaning: 189

Trash amount statistics:
count      189.000000
mean       541.158730
std       1465.665859
min         14.000000
25%        116.000000
50%        227.000000
75%        533.000000
max      18174.000000
Name: Trash amount, dtype: float64

🗺️ Creating Folium heatmap...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📊 Creating Plotly density map...



📅 Creating seasonal heatmap...



✅ All heatmaps created successfully!
You can interact with the maps above:
  - Zoom in/out with mouse wheel or buttons
  - Pan by clicking and dragging
  - Hover over areas to see details
  - Click markers for beach information


In [ ]:
import pandas as pd
import pydeck as pdk

# Load your data
df = pd.read_csv('cleaned_data.csv')
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

# Normalize trash amount for better visualization
df['normalized_trash'] = (df['Trash amount'] - df['Trash amount']. min()) / \
                          (df['Trash amount'].max() - df['Trash amount']. min())

# Create 3D heatmap layer
layer = pdk.Layer(
    'HeatmapLayer',
    data=df,
    get_position=['lon', 'lat'],
    get_weight='Trash amount',
    radiusPixels=60,
    intensity=1,
    threshold=0.05,
    opacity=0.8
)

# Set initial view
view_state = pdk.ViewState(
    latitude=41.8719,
    longitude=12.5674,
    zoom=5,
    pitch=0,
    bearing=0
)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style='mapbox://styles/mapbox/dark-v10'
)

r.to_html('pydeck_heatmap.html')

<IPython.core.display.Javascript object>

In [ ]:
# ============================================================================
# PYDECK HEATMAP - FIXED FOR GOOGLE COLAB
# ============================================================================
# PyDeck requires special configuration to work in Colab
# ============================================================================

# Install pydeck if not already installed
!pip install pydeck -q

import pandas as pd
import pydeck as pdk
from google.colab import drive

# Mount drive and load data
# drive.mount('/content/drive') # Commented out as file is directly in Colab environment
df = pd.read_csv('cleaned_data.csv') # Changed path to reflect file in Colab environment
df = df.dropna(subset=['lat', 'lon', 'Trash amount'])

print(f"✅ Data loaded:  {len(df)} rows")

# ----------------------------------------------------------------------------
# CRITICAL FIX: Enable PyDeck rendering in Colab
# ----------------------------------------------------------------------------
# This tells PyDeck to output HTML that Colab can display
pdk.settings.custom_libraries = []

# Normalize trash amount for better visualization (0-1 scale)
df['normalized_trash'] = (df['Trash amount'] - df['Trash amount'].min()) / \
                          (df['Trash amount'].max() - df['Trash amount']. min())

# ----------------------------------------------------------------------------
# Create HeatmapLayer
# ----------------------------------------------------------------------------
heatmap_layer = pdk.Layer(
    'HeatmapLayer',  # Layer type
    data=df,  # Your dataframe

    # Position:  MUST be [longitude, latitude] (note the order!)
    get_position=['lon', 'lat'],  # PyDeck uses [lon, lat], not [lat, lon]

    # Weight determines intensity at each point
    get_weight='Trash amount',  # Column to use for heatmap intensity

    # Visual parameters
    radius_pixels=60,  # Size of each point's influence (in pixels)
    intensity=1,  # Overall intensity multiplier (1-10)
    threshold=0.05,  # Minimum threshold to render (0-1)

    # Appearance
    opacity=0.8,  # Overall layer opacity
)

# ----------------------------------------------------------------------------
# Set the viewport (camera position)
# ----------------------------------------------------------------------------
view_state = pdk.ViewState(
    latitude=41.8719,  # Center latitude (Rome)
    longitude=12.5674,  # Center longitude (Rome)
    zoom=5.5,  # Zoom level (higher = closer)
    pitch=0,  # Tilt angle (0 = straight down, 60 = angled)
    bearing=0  # Rotation (0 = north up)
)

# ----------------------------------------------------------------------------
# Create the deck
# ----------------------------------------------------------------------------
r = pdk.Deck(
    layers=[heatmap_layer],  # List of layers to display
    initial_view_state=view_state,  # Camera position
    map_style='mapbox://styles/mapbox/dark-v10',  # Background style
    # Other options: 'light-v10', 'streets-v11', 'satellite-v9'

    # IMPORTANT: Set tooltip for interactivity
    tooltip={
        "html": "<b>Beach: </b> {Original_Beach_Name}<br/>"
                "<b>Trash: </b> {Trash amount} items<br/>"
                "<b>Date:</b> {Date of collection}",
        "style": {"backgroundColor": "steelblue", "color": "white"}
    }
)

# ----------------------------------------------------------------------------
# Display in Colab - METHOD 1: to_html()
# ----------------------------------------------------------------------------
# Save and display as HTML
r.to_html('pydeck_heatmap. html')

# Display using IFrame
from IPython.display import IFrame
IFrame(src='pydeck_heatmap.html', width=800, height=600)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cleaned_data.csv'